# Cleaning Data in SQL
Handling missing data, duplicate rows, and string cleaning.

## Missing values
I filter for records where the `unemployment_rate` is `NULL` and then `COUNT()` the number of rows. **163 missing values recorded**.

In [27]:
-- Identifying missing data --
SELECT COUNT(*) AS number_missing_unemployment_rates
FROM world.economies
WHERE unemployment_rate IS NULL;


,number_missing_unemployment_rates
0,163


### Filling missing values
I use `COALESCE()` to replace NULL values with the average unemployment rate (which is accessed via a subquery). `COALESCE()` works by returning the first argument if it is not null. If it is null, it returns the second argument, and so forth. If the `unemployment_rate` column is `NULL`, it returns the second argument, which is the average unemployment we calculate with a subquery.

In [28]:
-- Filling missing values --
WITH clean AS (
SELECT
	code,
	income_group,
	gdp_percapita,
	inflation_rate,
	total_investment,
    COALESCE(unemployment_rate, (SELECT AVG(unemployment_rate) FROM world.economies WHERE unemployment_rate IS NOT NULL)) AS filled_unemployment_rate,
	ROW_NUMBER() OVER(PARTITION BY code, unemployment_rate) AS row_number
FROM world.economies)

-- Discarding duplicate rows --
SELECT *
FROM clean
WHERE row_number = 1;

,code,income_group,gdp_percapita,inflation_rate,total_investment,filled_unemployment_rate,row_number
0,AFG,Low income,539.667,2.179,30.402,8.715097,1
1,AGO,Upper middle income,3599.270,14.480,14.433,8.715097,1
2,ALB,Upper middle income,4098.130,3.605,31.305,14.000000,1
3,ALB,Upper middle income,3943.220,1.896,24.598,17.100000,1
4,ARE,High income,34628.630,0.878,27.372,8.715097,1
...,...,...,...,...,...,...,...
293,YEM,Lower middle income,1266.790,11.175,11.661,8.715097,1
294,ZAF,Upper middle income,7361.940,4.264,19.513,24.875000,1
295,ZAF,Upper middle income,5721.150,4.575,20.892,25.350000,1
296,ZMB,Lower middle income,1456.160,8.500,29.878,8.715097,1


## Duplicate rows
To identify duplicate rows, I use `ROW_NUMBER()` to assign numbers to rows based on identical combinations. By choosing the `PARTITION` of the window function, I specify over code, unemployment_rate I want to look for duplicates.

I use `PARTITION BY` to assign row numbers based on the combination of country code and unemployment rate. Duplicate rows have a value of 2 or greater.

### Discarding duplicate rows
Removing duplicate rows is just as simple as identifying them. To do so, you simply need to change your filter to select `row_number`s with a value of 1.

## Invalid data
I search for rows where the `indep_year` contains a negative value. To do so, I convert the column to text using `::TEXT`, and then use `LIKE` and the pattern. The pattern I use searches for a minus sign (`-`), followed by any other characters (using the wildcard `%`).

In [29]:
SELECT indep_year
FROM world.countries
WHERE indep_year::TEXT LIKE '-%'

,indep_year
0,-1000
1,-660
2,-1523


I also use pattern matching to find rows with similar variants. I use a pattern to identify all rows with `Monarchy` in the `gov_form` column, useing the `%` wildcard characters to allow for words/whitespace on either side of the word.

In [30]:
SELECT DISTINCT name, gov_form
FROM world.countries
WHERE gov_form LIKE '%Monarchy%'

,name,gov_form
0,Sweden,Constitutional Monarchy
1,Tonga,Monarchy
2,Liechtenstein,Constitutional Monarchy
3,Jordan,Constitutional Monarchy
4,Solomon Islands,Constitutional Monarchy
5,Malaysia,"Constitutional Monarchy, Federation"
6,Saint Kitts and Nevis,Constitutional Monarchy
7,Norway,Constitutional Monarchy
8,Saint Vincent and the Grenadines,Constitutional Monarchy
9,Australia,"Constitutional Monarchy, Federation"


### Fixing invalid data
One way is to use a `CASE` statement to recategorize the data. I convert all `gov_form` rows that contain "Monarchy" to "Monarchy". The remaining entries are left as they are.

In [31]:
SELECT DISTINCT 
	name, 
    gov_form,
    CASE WHEN gov_form LIKE '%Monarchy%' THEN 'Monarchy' 
    ELSE gov_form END AS fixed_gov_form
FROM world.countries
WHERE gov_form LIKE '%Monarchy%'

,name,gov_form,fixed_gov_form
0,Saudi Arabia,Monarchy,Monarchy
1,United Kingdom,Constitutional Monarchy,Monarchy
2,Denmark,Constitutional Monarchy,Monarchy
3,Qatar,Monarchy,Monarchy
4,Brunei,Monarchy (Sultanate),Monarchy
5,Sweden,Constitutional Monarchy,Monarchy
6,Jamaica,Constitutional Monarchy,Monarchy
7,Antigua and Barbuda,Constitutional Monarchy,Monarchy
8,Samoa,Parlementary Monarchy,Monarchy
9,Australia,"Constitutional Monarchy, Federation",Monarchy


## Data types
In the query below, I retrieve each column and the data type for the `rental` table in the `dvdrentals` schema.

In [32]:
SELECT 
	column_name,
    data_type
FROM information_schema.columns
WHERE table_name = 'rental'

,column_name,data_type
0,rental_id,integer
1,rental_date,timestamp with time zone
2,inventory_id,integer
3,customer_id,integer
4,return_date,timestamp with time zone


### Converting data types
I convert two strings and two integers to different data types. The latter two columns produce identical results. The column `integer_to_text` converts the integer 16 to text using `CAST()`. The column `integer_to_text_with_operator` does the same with the cast operator `::`.

In [33]:
SELECT
	CAST('42' AS INTEGER) AS string_to_integer,
    CAST('2022-06-01' AS DATE) AS string_to_date,
    16::TEXT AS integer_to_text_with_operator

,string_to_integer,string_to_date,integer_to_text_with_operator
0,42,2022-06-01 00:00:00+00:00,16


### Converting date formats
Using `TO_CHAR()` to convert a given date to a provided format.

I use the short name of the month and the last two digits of the year to convert the precise rental date to a month_year column.

_Note: In Workspace, SQL queries are converted to pandas DataFrames. As a result, some formatting strings may result in Python automatically interpreting the result as a datetime and converting the date back to the original format._

In [34]:
SELECT 
  rental_id, 
  rental_date, 
  TO_CHAR(rental_date, 'Mon-YY') AS month_year
FROM dvdrentals.rental


,rental_id,rental_date,month_year
0,2,2005-05-25 02:54:33+00:00,May-05
1,3,2005-05-25 03:03:39+00:00,May-05
2,4,2005-05-25 03:04:41+00:00,May-05
3,5,2005-05-25 03:05:21+00:00,May-05
4,6,2005-05-25 03:08:07+00:00,May-05
...,...,...,...
16039,16046,2005-08-24 02:26:47+00:00,Aug-05
16040,16047,2005-08-24 02:42:48+00:00,Aug-05
16041,16048,2005-08-24 02:43:07+00:00,Aug-05
16042,16049,2005-08-24 02:50:12+00:00,Aug-05
